In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


In [2]:
data = pd.read_csv("Data/StudentsPerformance.csv")

In [3]:
data.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
X_train,X_test,y_train,y_test = train_test_split(data.drop(columns=["math score"]),data["math score"],train_size=0.8)

In [5]:
X_train.shape

(800, 7)

In [6]:
cat_colums = X_train.select_dtypes(include="object").columns
num_columns = X_train.select_dtypes(exclude="object").columns

In [7]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

ohe = OneHotEncoder()
sc = StandardScaler()

preproccesor = ColumnTransformer(
    [
    ("onehot",ohe,cat_colums),
    ("standard_scaler",sc,num_columns)
    ]
)

In [8]:
x_train = preproccesor.fit_transform(X_train)

In [9]:
x_train

array([[1.        , 0.        , 1.        , ..., 0.        , 1.12174403,
        0.81492095],
       [1.        , 0.        , 0.        , ..., 0.        , 0.70816608,
        0.88044882],
       [1.        , 0.        , 0.        , ..., 0.        , 2.1556889 ,
        2.12547831],
       ...,
       [0.        , 1.        , 0.        , ..., 0.        , 0.36351779,
        0.22517014],
       [0.        , 1.        , 0.        , ..., 1.        , 0.57030676,
        0.35622587],
       [0.        , 1.        , 0.        , ..., 1.        , 0.5013771 ,
        0.22517014]], shape=(800, 19))

In [10]:
x_test = preproccesor.transform(X_test)

In [11]:
x_test.shape

(200, 19)

In [12]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [15]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.2846
- Mean Absolute Error: 4.2288
- R2 Score: 0.8799
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4986
- Mean Absolute Error: 4.3440
- R2 Score: 0.8561


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.4945
- Mean Absolute Error: 5.0868
- R2 Score: 0.8186
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5240
- Mean Absolute Error: 5.2699
- R2 Score: 0.7975


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.2848
- Mean Absolute Error: 4.2279
- R2 Score: 0.8799
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.5002
- Mean Absolute Error: 4.3438
- R2 Score: 0.8560


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.8123
- Mean Absolute Error: 4.5483
- R2 Score: 0.8547
-----------------------

In [16]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,0.856127
2,Ridge,0.856042
7,CatBoosting Regressor,0.839953
5,Random Forest Regressor,0.831107
8,AdaBoost Regressor,0.810986
1,Lasso,0.797467
6,XGBRegressor,0.791085
3,K-Neighbors Regressor,0.765738
4,Decision Tree,0.665356
